<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Leveraging Apache Spark for Smart Building HVAC Monitoring

**Estimated time needed: 30 minutes**

### Objectives

After completing this lab, you will be able to:

- Explain the distributed architecture of Spark in the context of smart building monitoring
- Simulate real-time sensor data for HVAC systems in a building
- Perform SQL queries to detect critical environmental conditions and calculate average readings
- Determine the aggregated results to the console for immediate insights into room conditions


## Background
Smart Building Solutions, Inc. specializes in optimizing HVAC (heating, ventilation, and air conditioning) systems to enhance comfort and energy efficiency in commercial buildings. By monitoring temperature and humidity levels in real-time across various rooms, the company aims to ensure optimal indoor conditions and preemptively address potential HVAC issues.

With a continuous influx of sensor data, Smart Building Solutions needs to process and analyze this data in real-time to maintain the quality of the indoor environment.

## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


25/08/14 17:02:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [4]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [5]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [6]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [7]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)

### Output the results to the console:

Display the results from each query in real-time.


In [8]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|69.61989011928709|2025-08-14 17:11:...|
|      0|       15.0|66.23844519185347|2025-08-14 17:11:...|
|      0|       15.0|65.79174677384947|2025-08-14 17:11:...|
|      0|       15.0|69.45266016207873|2025-08-14 17:11:...|
+-------+-----------+-----------------+--------------------+



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



[Stage 4:>(191 + 8) / 200][Stage 5:>    (0 + 0) / 8][Stage 6:>    (0 + 0) / 8]8]

-------------------------------------------
Batch: 2
-------------------------------------------


...

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 67.02916930540553|2025-08-14 17:11:...|
|      0|       15.0| 46.06628876498657|2025-08-14 17:11:...|
|      0|       15.0| 52.18893494419664|2025-08-14 17:11:...|
|      0|       15.0|  49.8140194496045|2025-08-14 17:11:...|
|      0|       15.0|58.791272254331616|2025-08-14 17:12:...|
|      0|       15.0| 46.25021520125162|2025-08-14 17:12:...|
|      0|       15.0| 59.06858251095331|2025-08-14 17:11:...|
|      0|       15.0| 51.98692694257697|2025-08-14 17:11:...|
|      0|       15.0| 63.64622315114896|2025-08-14 17:11:...|
|      0|       15.0| 41.56241830266276|2025-08-14 17:11:...|
|      0|       15.0| 68.04726497207699|2025-08-14 17:12:...|
|      0|       15.0| 43.7162411849

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 48.72659189867497|2025-08-14 17:12:...|
|      0|       15.0| 45.78018518869703|2025-08-14 17:12:...|
|      0|       15.0| 50.49332706834262|2025-08-14 17:12:...|
|      0|       15.0| 64.66338887283152|2025-08-14 17:12:...|
|      0|       15.0| 54.86179033537222|2025-08-14 17:12:...|
|      0|       15.0|48.815062210485486|2025-08-14 17:12:...|
|      0|       15.0| 40.38189790652731|2025-08-14 17:12:...|
|      0|       15.0|46.555174997542814|2025-08-14 17:12:...|
|      0|       15.0|45.096105357074705|2025-08-14 17:12:...|
|      0|       15.0| 69.70599271318883|2025-08-14 17:12:...|
|      0|       15.0| 66.75480101485564|2025-08-14 17:12:...|
+-------+-----------+--------------

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 44.71366285949736|2025-08-14 17:12:...|
|      0|       15.0| 67.68735705489699|2025-08-14 17:12:...|
|      0|       15.0| 55.94762490998404|2025-08-14 17:12:...|
|      0|       15.0| 54.03547735974191|2025-08-14 17:12:...|
|      0|       15.0|43.395355667992874|2025-08-14 17:12:...|
|      0|       15.0| 48.24297828511017|2025-08-14 17:12:...|
|      0|       15.0| 57.21172077906258|2025-08-14 17:12:...|
|      0|       15.0| 45.48936693256907|2025-08-14 17:12:...|
|      0|       15.0|52.290812474007865|2025-08-14 17:12:...|
|      0|       15.0| 55.01870805737277|2025-08-14 17:12:...|
+-------+-----------+------------------+--------------------+

----------------------------------

-------------------------------------------
Batch: 6
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|   48.854124701852|2025-08-14 17:12:...|
|      0|       15.0|57.706603179260284|2025-08-14 17:13:...|
|      0|       15.0| 62.02044822361335|2025-08-14 17:13:...|
|      0|       15.0| 62.56263348869206|2025-08-14 17:13:...|
|      0|       15.0| 65.37086232057095|2025-08-14 17:13:...|
|      0|       15.0| 57.05648213619661|2025-08-14 17:13:...|
|      0|       15.0| 52.12777897292227|2025-08-14 17:13:...|
|      0|       15.0|49.021036548713276|2025-08-14 17:13:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+----

[Stage 19:(103 + 8) / 200][Stage 20:>   (0 + 0) / 8][Stage 21:>   (0 + 0) / 8]8]

### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [ ]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


[Stage 19:(109 + 8) / 200][Stage 20:>   (0 + 0) / 8][Stage 21:>   (0 + 0) / 8]

********Critical Temperature Values*******


-------------------------------------------
Batch: 7
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 58.59226273675866|2025-08-14 17:13:...|
|      0|       15.0| 52.55877893368741|2025-08-14 17:13:...|
|      0|       15.0| 42.61827971635709|2025-08-14 17:13:...|
|      0|       15.0| 51.97972215235703|2025-08-14 17:13:...|
|      0|       15.0| 56.96551004881708|2025-08-14 17:13:...|
|      0|       15.0| 43.66142195378562|2025-08-14 17:13:...|
|      0|       15.0|44.661288454332805|2025-08-14 17:13:...|
|      0|       15.0| 56.21879748870037|2025-08-14 17:13:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+------------------+------------------+--------------

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 48.24822603011676|2025-08-14 17:13:...|
|      0|       15.0| 66.45309376876844|2025-08-14 17:13:...|
|      0|       15.0|42.693683305890296|2025-08-14 17:13:...|
|      0|       15.0| 63.94577864832823|2025-08-14 17:13:...|
|      0|       15.0| 46.11175552037551|2025-08-14 17:13:...|
|      0|       15.0| 68.08442916147509|2025-08-14 17:13:...|
|      0|       15.0| 62.12017232929455|2025-08-14 17:13:...|
|      0|       15.0| 59.70017234678234|2025-08-14 17:13:...|
|      0|       15.0|  57.3180932900214|2025-08-14 17:13:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------

-------------------------------------------
Batch: 9
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 44.80912217139009|2025-08-14 17:13:...|
|      0|       15.0| 56.54090082923595|2025-08-14 17:14:...|
|      0|       15.0| 64.92385509289956|2025-08-14 17:13:...|
|      0|       15.0| 61.12623939223414|2025-08-14 17:14:...|
|      0|       15.0| 50.46355511072296|2025-08-14 17:13:...|
|      0|       15.0| 67.71385277403802|2025-08-14 17:13:...|
|      0|       15.0|51.246154992729906|2025-08-14 17:14:...|
|      0|       15.0| 68.04880139779989|2025-08-14 17:13:...|
|      0|       15.0| 66.35723009766566|2025-08-14 17:13:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------

-------------------------------------------
Batch: 10
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 58.76056468316311|2025-08-14 17:14:...|
|      0|       15.0| 51.71946877276347|2025-08-14 17:14:...|
|      0|       15.0| 52.91256506529383|2025-08-14 17:14:...|
|      0|       15.0| 63.42106500439564|2025-08-14 17:14:...|
|      0|       15.0|48.249994510397315|2025-08-14 17:14:...|
|      0|       15.0| 69.08189012836401|2025-08-14 17:14:...|
|      0|       15.0|  67.2832966102614|2025-08-14 17:14:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               15|
|      3| 

[Stage 31:(196 + 4) / 200][Stage 32:=>  (3 + 4) / 8][Stage 33:>   (0 + 0) / 8]8]

-------------------------------------------
Batch: 11
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|46.23237711106355|2025-08-14 17:14:...|
|      0|       15.0|55.09296848880531|2025-08-14 17:14:...|
|      0|       15.0|45.40997736195316|2025-08-14 17:14:...|
|      0|       15.0|64.39575290189995|2025-08-14 17:14:...|
|      0|       15.0|55.99116383562182|2025-08-14 17:14:...|
|      0|       15.0|64.40037737466868|2025-08-14 17:14:...|
|      0|       15.0|68.74077800043466|2025-08-14 17:14:...|
|      0|       15.0|68.96856858299063|2025-08-14 17:14:...|
|      0|       15.0|51.43722147514218|2025-08-14 17:14:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+--

-------------------------------------------
Batch: 12
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 46.50867034200911|2025-08-14 17:14:...|
|      0|       15.0|55.402465137430546|2025-08-14 17:14:...|
|      0|       15.0| 44.06509782011818|2025-08-14 17:14:...|
|      0|       15.0|  61.5476322368799|2025-08-14 17:14:...|
|      0|       15.0|46.149907878078196|2025-08-14 17:14:...|
|      0|       15.0| 41.66366526038296|2025-08-14 17:14:...|
|      0|       15.0| 58.74414353505723|2025-08-14 17:14:...|
|      0|       15.0| 42.29682130625985|2025-08-14 17:14:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+---

[Stage 37:(172 + 8) / 200][Stage 38:>   (0 + 0) / 8][Stage 39:>   (0 + 0) / 8]]]

### Conclusion
In this lab, you explored the use of Apache Spark in smart building monitoring, particularly focusing on HVAC (heating, ventilation, and air conditioning) systems. You now understand the Spark's distributed architecture. You also understand how to simulate real-time sensor data for temperature and humidity, execute SQL queries to identify critical environmental conditions, and output aggregated results for immediate insights.


## Author(s)

Lakshmi Holla

## Other Contributors
Malika Singla
